In [ ]:
import logging
import asyncio
import polars as pl
import plotly.graph_objects as go

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.analytics.visualizations.charts import Study

from tastytrade.providers.market import MarketDataProvider
from tastytrade.messaging.models.events import CandleEvent

from tastytrade.analytics.indicators.momentum import hull

# Show all rows in pandas DataFrames
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(None)

logging.getLogger().handlers.clear()

TEST = True

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

In [3]:
influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials().url, token=influx_user.token, org=influx_user.org
)

In [4]:
streamer = MarketDataProvider(dxlink, influxdb)

start = datetime(2025, 2, 21, 9) + timedelta(hours=5)
stop = datetime(2025, 2, 21, 16) + timedelta(hours=5)

candles: pl.DataFrame = streamer.download(
    symbol="SPX{=5m}",
    event_type="CandleEvent",
    start=start,
    stop=stop,
    debug_mode=True,
)

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol="SPX{=d}",
            event_type="CandleEvent",
            start=start + timedelta(days=-5),
            stop=start,
            debug_mode=True,
        )
        .filter(pl.col("time") < start.date())
        .tail(1)
        .to_dicts()[0]
    )
)

In [ ]:
chart_config = {"height": 800, "xaxis_rangeslider_visible": True}

fig = go.Figure(
    data=[
        go.Candlestick(
            x=candles["time"].dt.replace_time_zone("UTC").dt.convert_time_zone("America/New_York"),
            open=candles["open"],
            high=candles["high"],
            low=candles["low"],
            close=candles["close"],
        )
    ]
)
fig.update_layout(**chart_config)
fig.show()

In [ ]:
hull(input_df=candles).head()

In [6]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,  # The hull function
    params={"length": 20},  # Parameters for hull function
    plot_params={
        "colors": {"Up": "#01FFFF", "Down": "#FF66FE"},  # Green for uptrend  # Red for downtrend
        "width": 2,
    },
    value_column="HMA",
    color_column="HMA_color",
)

In [9]:
candles = candles.with_columns(
    [
        pl.col("close").rolling_mean(window_size=20).alias("MA20"),
        pl.col("close").rolling_mean(window_size=5).alias("MA5"),
    ]
)

In [7]:
import polars as pl
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def ema_with_seed(values: np.ndarray, length: int, seed: float) -> np.ndarray:
    alpha = 2.0 / (length + 1.0)
    out = np.zeros_like(values, dtype=float)
    if len(values) == 0:
        return out

    # Seed the first EMA
    out[0] = alpha * values[0] + (1 - alpha) * seed
    # Compute forward
    for i in range(1, len(values)):
        out[i] = alpha * values[i] + (1 - alpha) * out[i - 1]

    return out


def compute_macd_single_day(
    df_day: pl.DataFrame,
    prior_close: float,
    fast_length: int = 12,
    slow_length: int = 26,
    macd_length: int = 9,
) -> pl.DataFrame:
    # Sort by time just to be safe
    df_day = df_day.sort("time")
    # Convert 'close' to numpy
    close_np = df_day["close"].to_numpy()

    # 1) Fast EMA (seeded by prior_close)
    ema_fast = ema_with_seed(close_np, fast_length, seed=prior_close)
    # 2) Slow EMA (seeded by prior_close)
    ema_slow = ema_with_seed(close_np, slow_length, seed=prior_close)
    # MACD Value line
    value = ema_fast - ema_slow

    # 3) Signal line = EMA of Value (seed with 0.0 or some small guess).
    #    If you prefer to seed it with prior day's final MACD, you can do so;
    #    but the simplest is to seed with 0.0 or the difference from prior_close.
    signal_seed = 0.0
    ema_signal = ema_with_seed(value, macd_length, seed=signal_seed)

    # 4) Histogram
    diff = value - ema_signal

    # Attach them as new columns
    df_res = df_day.with_columns(
        [
            pl.Series("Value", value),
            pl.Series("Avg", ema_signal),
            pl.Series("Diff", diff),
        ]
    )
    return df_res


def plot_macd(df: pl.DataFrame):
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.4, 0.2]
    )

    # --- Top subplot: Candlestick chart ---
    fig.add_trace(
        go.Candlestick(
            x=df["time"],
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
            name="Price",
            showlegend=False,
        ),
        row=1,
        col=1,
    )

    # --- Bottom subplot: MACD Value, Signal, Histogram ---
    fig.add_trace(
        go.Scatter(
            x=df["time"],
            y=df["Value"],
            mode="lines",
            name="MACD (Value)",
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=df["time"],
            y=df["Avg"],
            mode="lines",
            name="Signal (Avg)",
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    # Histogram bars
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df["Diff"],
            name="Histogram (Diff)",
            marker_color=["green" if val >= 0 else "red" for val in df["Diff"]],
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    # Zero line
    fig.add_shape(
        type="line",
        x0=df["time"][0],
        x1=df["time"][-1],
        y0=0,
        y1=0,
        line=dict(color="black", width=1, dash="dot"),
        xref="x2",
        yref="y2",
    )

    fig.update_layout(
        title="Single-Day MACD",
        xaxis2_title="Time",
        yaxis_title="Price",
        yaxis2_title="MACD",
        showlegend=True,
        height=800,
        xaxis_rangeslider_visible=False,
    )

    fig.show()

In [ ]:
# Compute MACD columns
df_macd = compute_macd_single_day(
    candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)

# Plot with Plotly
plot_macd(df_macd)

In [ ]:
from IPython.display import Image

Image(filename="/workspace/devtools/images/macd_sample.png", width=1000)